In [1]:
import psycopg2
from helper import normalise_price, normalise_area, normalise_address

In [17]:
conn = psycopg2.connect(database = "postgres", 
                        user = "postgres.iteuczlaiijtvkpplsms", 
                        host= 'aws-0-ap-southeast-1.pooler.supabase.com',
                        password = "aUbEUkOgw23zCDdc",
                        port = 6543)
conn.autocommit = True

In [28]:
cur = conn.cursor()
cur.execute(
    """
    SELECT * FROM cos40005_property 
    WHERE id IS NOT NULL
        AND title IS NOT NULL AND title <> ''
        AND address IS NOT NULL AND address <> ''
        AND price IS NOT NULL AND price <> ''
        AND area IS NOT NULL AND area <> '';
    """)
rows = cur.fetchall()
conn.commit()
cur.close()

In [29]:
len(rows)

In [15]:
conn.close()

In [32]:
f = open("./output.csv", "w")
f.write("id,price,is_rent,area,province,district,ward,floor,bedroom,toilet\n")
for row in rows:
    id = row[0]
    norm_price = normalise_price(row[3])
    price = norm_price["price"]
    is_rent = norm_price["is_rent"]
    area = normalise_area(row[4])
    norm_addr = normalise_address(row[2])
    province = norm_addr["province"]
    district = norm_addr["district"]
    ward = norm_addr["ward"]
    floor = row[5]
    bedroom = row[6]
    toilet = row[7]
    f.write(','.join([id,price,is_rent,area,province,district,ward,floor,bedroom,toilet]) + "\n")
f.close()

In [35]:
import pandas as pd


df = pd.DataFrame(pd.read_csv("./output.csv"))
df_no_duplicates = df.drop_duplicates(subset=df.columns.difference(['id']))
df_no_duplicates.drop('title', axis=1, inplace=True)
df_no_duplicates.to_csv('./output_cleaned.csv', index=False)